<h1><b><u>Projeto IA - Connect 4</u></b></h1>

<h3>Membros do Grupo:</h3>
<p>Maximiliano Vítor Phillips e Sá (up202305979)</p>
<p>Orlando Miguel Carvalho-Soares (up202303606)</p>
<p>Rui Jorge Pereira Rua (up202305259)</p>